In [4]:
import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [5]:
from minsearch import Index

index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [6]:
from typing import Any, Dict, List, TypedDict

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str


def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )


In [7]:
file_index = {}

for doc in parsed_data:
    filename = doc['filename']
    file_index[filename] = doc


In [9]:
from typing import Optional

def read_file(filename: str) -> Optional[str]:
    """
    Retrieve the content of a file from the repository.

    Args:
        filename (str): The name or path of the file to read.

    Returns:
        Optional[str]: The file content as a string if the file exists;
        otherwise, returns None.
    """
    if filename in file_index:
        return file_index[filename]['content']
    return None


In [10]:
instructions = """
You are an assistant that helps improve and generate high-quality documentation for the project.

You have access to the following tools:
- search — Use this to explore topics in depth. Make multiple search calls if needed to gather comprehensive information.
- read_file — Use this when code snippets are missing or when you need to retrieve the full content of a file for context.

Critical Rule

Before generating or finalizing any code example or technical explanation, you must always call `read_file`
to cross-check the correctness of the code.
Do not rely solely on search results or assumptions — always verify by reading the actual file content.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

When answering a question:
1. Provide file references for all source materials.  
   Use this format:  
   [{filename}](https://github.com/evidentlyai/docs/blob/main/{filename})
2. If the topic is covered in multiple documents, cite all relevant sources.
3. Include code examples whenever they clarify or demonstrate the concept.
4. Be concise, accurate, and helpful — focus on clarity and usability for developers.
5. If documentation is missing or unclear, infer from context and note that explicitly.

Example Citation

See the full implementation in [metrics/api_reference.md](https://github.com/evidentlyai/docs/blob/main/metrics/api_reference.md).
""".strip()


In [12]:
import agents
from dotenv import load_dotenv
# from openai import OpenAI
# openai_client = OpenAI()
load_dotenv()



True

In [13]:
agent_tools = [search, read_file]

from pydantic_ai import Agent

agent = Agent(
    name="docs_agent",
    instructions=instructions,
    tools=agent_tools,
    model='gpt-4o-mini'
)


In [14]:
results = await agent.run(user_prompt='how do I run llm as a judge evals?',
                         # message_history=results.all_messages()
                         )


In [15]:
print(results.output)

To run an LLM (Large Language Model) as a judge for evaluations, you can follow these general steps based on the tutorial found in the [LLM as a judge](https://github.com/evidentlyai/docs/blob/main/examples/LLM_judge.mdx):

### 1. Prerequisites
Ensure you have:
- Basic knowledge of Python.
- An OpenAI API key for using the LLM evaluator.

### 2. Install Required Libraries
You need to install the `evidently` library, which provides tools for evaluating models including LLMs.

```bash
pip install evidently
```

### 3. Import the Required Modules
Begin by importing necessary libraries in your Python environment:

```python
import pandas as pd
import numpy as np

from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently import BinaryClassification
from evidently.descriptors import *
from evidently.presets import TextEvals, ClassificationPreset
from evidently.metrics import *
from evidently.llm.templates import BinaryClassificationPromptT

In [17]:
messages = results.all_messages()

In [27]:
for message in results.new_messages():
    print(message.kind)

request
response
request
response
request
response


In [21]:
results.usage()


RunUsage(input_tokens=11566, cache_read_tokens=3072, output_tokens=932, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=3, tool_calls=2)

In [28]:
# from toyaikit.pricing import PricingConfig
# pricing = PricingConfig()

# usage = results.usage()

# pricing.calculate_cost(
#     model=agent.model.model_name,
#     input_tokens=usage.input_tokens,
#     output_tokens=usage.output_tokens
# )
